In [2]:
import pygame
import time
import sys

# Initialize pygame before loading images
pygame.init()

board = [['  ' for i in range(8)] for i in range(8)]

class Piece:
    def __init__(self, team, type, image, killable=False):
        self.team = team
        self.type = type
        self.killable = killable
        self.image = image

# Create pieces
bp = Piece('b', 'p', 'b_pawn.png')
wp = Piece('w', 'p', 'w_pawn.png')
bk = Piece('b', 'k', 'b_king.png')
wk = Piece('w', 'k', 'w_king.png')
br = Piece('b', 'r', 'b_rook.png')
wr = Piece('w', 'r', 'w_rook.png')
bb = Piece('b', 'b', 'b_bishop.png')
wb = Piece('w', 'b', 'w_bishop.png')
bq = Piece('b', 'q', 'b_queen.png')
wq = Piece('w', 'q', 'w_queen.png')
bkn = Piece('b', 'kn', 'b_knight.png')
wkn = Piece('w', 'kn', 'w_knight.png')

# Load images after pygame.init()
try:
    starting_order = {
        (0, 0): pygame.image.load(br.image), (1, 0): pygame.image.load(bkn.image),
        (2, 0): pygame.image.load(bb.image), (3, 0): pygame.image.load(bk.image),
        (4, 0): pygame.image.load(bq.image), (5, 0): pygame.image.load(bb.image),
        (6, 0): pygame.image.load(bkn.image), (7, 0): pygame.image.load(br.image),
        (0, 1): pygame.image.load(bp.image), (1, 1): pygame.image.load(bp.image),
        (2, 1): pygame.image.load(bp.image), (3, 1): pygame.image.load(bp.image),
        (4, 1): pygame.image.load(bp.image), (5, 1): pygame.image.load(bp.image),
        (6, 1): pygame.image.load(bp.image), (7, 1): pygame.image.load(bp.image),

        (0, 2): None, (1, 2): None, (2, 2): None, (3, 2): None,
        (4, 2): None, (5, 2): None, (6, 2): None, (7, 2): None,
        (0, 3): None, (1, 3): None, (2, 3): None, (3, 3): None,
        (4, 3): None, (5, 3): None, (6, 3): None, (7, 3): None,
        (0, 4): None, (1, 4): None, (2, 4): None, (3, 4): None,
        (4, 4): None, (5, 4): None, (6, 4): None, (7, 4): None,
        (0, 5): None, (1, 5): None, (2, 5): None, (3, 5): None,
        (4, 5): None, (5, 5): None, (6, 5): None, (7, 5): None,

        (0, 6): pygame.image.load(wp.image), (1, 6): pygame.image.load(wp.image),
        (2, 6): pygame.image.load(wp.image), (3, 6): pygame.image.load(wp.image),
        (4, 6): pygame.image.load(wp.image), (5, 6): pygame.image.load(wp.image),
        (6, 6): pygame.image.load(wp.image), (7, 6): pygame.image.load(wp.image),
        (0, 7): pygame.image.load(wr.image), (1, 7): pygame.image.load(wkn.image),
        (2, 7): pygame.image.load(wb.image), (3, 7): pygame.image.load(wk.image),
        (4, 7): pygame.image.load(wq.image), (5, 7): pygame.image.load(wb.image),
        (6, 7): pygame.image.load(wkn.image), (7, 7): pygame.image.load(wr.image),
    }
except pygame.error as e:
    print(f"Error loading images: {e}")
    # Create blank surfaces as fallback
    blank = pygame.Surface((50, 50))
    blank.fill((255, 255, 255))
    starting_order = {pos: blank for pos in starting_order if starting_order[pos] is not None}

def create_board(board):
    board[0] = [Piece('b', 'r', 'b_rook.png'), Piece('b', 'kn', 'b_knight.png'), Piece('b', 'b', 'b_bishop.png'),
               Piece('b', 'q', 'b_queen.png'), Piece('b', 'k', 'b_king.png'), Piece('b', 'b', 'b_bishop.png'),
               Piece('b', 'kn', 'b_knight.png'), Piece('b', 'r', 'b_rook.png')]

    board[7] = [Piece('w', 'r', 'w_rook.png'), Piece('w', 'kn', 'w_knight.png'), Piece('w', 'b', 'w_bishop.png'),
               Piece('w', 'q', 'w_queen.png'), Piece('w', 'k', 'w_king.png'), Piece('w', 'b', 'w_bishop.png'),
               Piece('w', 'kn', 'w_knight.png'), Piece('w', 'r', 'w_rook.png')]

    for i in range(8):
        board[1][i] = Piece('b', 'p', 'b_pawn.png')
        board[6][i] = Piece('w', 'p', 'w_pawn.png')
    return board

def on_board(position):
    if 0 <= position[0] < 8 and 0 <= position[1] < 8:
        return True
    return False

def convert_to_readable(board):
    output = ''
    for i in board:
        for j in i:
            try:
                output += j.team + j.type + ', '
            except:
                output += j + ', '
        output += '\n'
    return output

def deselect():
    for row in range(len(board)):
        for column in range(len(board[0])):
            if board[row][column] == 'x ':
                board[row][column] = '  '
            else:
                try:
                    board[row][column].killable = False
                except:
                    pass
    return convert_to_readable(board)

def highlight(board):
    highlighted = []
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 'x ':
                highlighted.append((j, i))  # Note: Swapped i and j to match display coordinates
            else:
                try:
                    if board[i][j].killable:
                        highlighted.append((j, i))  # Note: Swapped i and j
                except:
                    pass
    return highlighted

def check_team(moves, index):
    row, col = index
    if board[row][col] == '  ':
        return False
    if moves % 2 == 0:
        return board[row][col].team == 'w'
    else:
        return board[row][col].team == 'b'

def select_moves(piece, index, moves):
    if not check_team(moves, index):
        return []
    
    row, col = index
    if piece.type == 'p':
        if piece.team == 'b':
            return highlight(pawn_moves_b(index))
        else:
            return highlight(pawn_moves_w(index))
    elif piece.type == 'k':
        return highlight(king_moves(index))
    elif piece.type == 'r':
        return highlight(rook_moves(index))
    elif piece.type == 'b':
        return highlight(bishop_moves(index))
    elif piece.type == 'q':
        return highlight(queen_moves(index))
    elif piece.type == 'kn':
        return highlight(knight_moves(index))
    return []

def pawn_moves_b(index):
    row, col = index
    if row == 1:
        if board[row + 2][col] == '  ' and board[row + 1][col] == '  ':
            board[row + 2][col] = 'x '
    bottom3 = [[row + 1, col + i] for i in range(-1, 2)]

    for positions in bottom3:
        if on_board(positions):
            r, c = positions
            if bottom3.index(positions) % 2 == 0:
                try:
                    if board[r][c].team != 'b':
                        board[r][c].killable = True
                except:
                    pass
            else:
                if board[r][c] == '  ':
                    board[r][c] = 'x '
    return board

def pawn_moves_w(index):
    row, col = index
    if row == 6:
        if board[row - 2][col] == '  ' and board[row - 1][col] == '  ':
            board[row - 2][col] = 'x '
    top3 = [[row - 1, col + i] for i in range(-1, 2)]

    for positions in top3:
        if on_board(positions):
            r, c = positions
            if top3.index(positions) % 2 == 0:
                try:
                    if board[r][c].team != 'w':
                        board[r][c].killable = True
                except:
                    pass
            else:
                if board[r][c] == '  ':
                    board[r][c] = 'x '
    return board

def king_moves(index):
    row, col = index
    for y in range(3):
        for x in range(3):
            if on_board((row - 1 + y, col - 1 + x)):
                r, c = row - 1 + y, col - 1 + x
                if board[r][c] == '  ':
                    board[r][c] = 'x '
                else:
                    if board[r][c].team != board[row][col].team:
                        board[r][c].killable = True
    return board

def rook_moves(index):
    row, col = index
    directions = [
        [(row + i, col) for i in range(1, 8 - row)],  # Down
        [(row - i, col) for i in range(1, row + 1)],   # Up
        [(row, col + i) for i in range(1, 8 - col)],   # Right
        [(row, col - i) for i in range(1, col + 1)]    # Left
    ]

    for direction in directions:
        for r, c in direction:
            if on_board((r, c)):
                if board[r][c] == '  ':
                    board[r][c] = 'x '
                else:
                    if board[r][c].team != board[row][col].team:
                        board[r][c].killable = True
                    break
    return board

def bishop_moves(index):
    row, col = index
    directions = [
        [(row + i, col + i) for i in range(1, 8)],
        [(row + i, col - i) for i in range(1, 8)],
        [(row - i, col + i) for i in range(1, 8)],
        [(row - i, col - i) for i in range(1, 8)]
    ]

    for direction in directions:
        for r, c in direction:
            if on_board((r, c)):
                if board[r][c] == '  ':
                    board[r][c] = 'x '
                else:
                    if board[r][c].team != board[row][col].team:
                        board[r][c].killable = True
                    break
    return board

def queen_moves(index):
    board = rook_moves(index)
    board = bishop_moves(index)
    return board

def knight_moves(index):
    row, col = index
    moves = [
        (row + 2, col + 1), (row + 2, col - 1),
        (row - 2, col + 1), (row - 2, col - 1),
        (row + 1, col + 2), (row + 1, col - 2),
        (row - 1, col + 2), (row - 1, col - 2)
    ]

    for r, c in moves:
        if on_board((r, c)):
            if board[r][c] == '  ':
                board[r][c] = 'x '
            else:
                if board[r][c].team != board[row][col].team:
                    board[r][c].killable = True
    return board

WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("Chess")

WHITE = (255, 255, 255)
GREY = (128, 128, 128)
YELLOW = (204, 204, 0)
BLUE = (50, 255, 255)
BLACK = (0, 0, 0)

class Node:
    def __init__(self, row, col, width):
        self.row = row
        self.col = col
        self.x = int(row * width)
        self.y = int(col * width)
        self.colour = WHITE
        self.occupied = None

    def draw(self, WIN):
        pygame.draw.rect(WIN, self.colour, (self.x, self.y, WIDTH / 8, WIDTH / 8))

    def setup(self, WIN):
        if (self.row, self.col) in starting_order and starting_order[(self.row, self.col)]:
            WIN.blit(starting_order[(self.row, self.col)], (self.x, self.y))

def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            node = Node(j, i, gap)
            grid[i].append(node)
            if (i + j) % 2 == 1:
                grid[i][j].colour = GREY
    return grid

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows + 1):
        pygame.draw.line(win, BLACK, (0, i * gap), (width, i * gap))
        pygame.draw.line(win, BLACK, (i * gap, 0), (i * gap, width))

def update_display(win, grid, rows, width):
    for row in grid:
        for spot in row:
            spot.draw(win)
            spot.setup(win)
    draw_grid(win, rows, width)
    pygame.display.update()

def Find_Node(pos, WIDTH):
    interval = WIDTH / 8
    y, x = pos
    rows = int(x // interval)
    columns = int(y // interval)
    return rows, columns

def display_potential_moves(positions, grid):
    for x, y in positions:
        grid[y][x].colour = BLUE  # Note: Swapped x and y to match grid coordinates

def Do_Move(OriginalPos, FinalPosition, WIN):
    starting_order[FinalPosition] = starting_order[OriginalPos]
    starting_order[OriginalPos] = None

def remove_highlight(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if (i + j) % 2 == 0:
                grid[i][j].colour = WHITE
            else:
                grid[i][j].colour = GREY
    return grid

create_board(board)

def main(WIN, WIDTH):
    moves = 0
    selected = False
    piece_to_move = []
    grid = make_grid(8, WIDTH)
    
    while True:
        pygame.time.delay(50)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                col, row = Find_Node(pos, WIDTH)  # Note: Swapped to match board coordinates
                
                if not selected:
                    try:
                        if board[row][col] != '  ' and check_team(moves, (row, col)):
                            possible = select_moves(board[row][col], (row, col), moves)
                            display_potential_moves(possible, grid)
                            piece_to_move = (row, col)
                            selected = True
                    except Exception as e:
                        print(f"Selection error: {e}")
                        piece_to_move = []
                        selected = False
                else:
                    try:
                        # Check if the move is valid
                        if (col, row) in highlight(board) or (board[row][col] != '  ' and board[row][col].killable):
                            # Perform the move
                            old_row, old_col = piece_to_move
                            board[row][col] = board[old_row][old_col]
                            board[old_row][old_col] = '  '
                            deselect()
                            remove_highlight(grid)
                            Do_Move((old_col, old_row), (col, row), WIN)
                            moves += 1
                            print(convert_to_readable(board))
                        else:
                            deselect()
                            remove_highlight(grid)
                    except Exception as e:
                        print(f"Move error: {e}")
                    selected = False

        update_display(WIN, grid, 8, WIDTH)

if __name__ == "__main__":
    main(WIN, WIDTH)

pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Saffa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
